### Import All Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures,OneHotEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix,classification_report
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
import scipy.stats as stats
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint, uniform

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import warnings
warnings.filterwarnings("ignore")

### Load original datasets - 6

In [2]:
targetdf = pd.read_csv('C:/Users/u1427957/OneDrive - University of Utah/MyWorks_2022/Portfolio-Projects/Humana/data/new_data/target_train.csv')
medclmsdf = pd.read_csv('C:/Users/u1427957/OneDrive - University of Utah/MyWorks_2022/Portfolio-Projects/Humana/data/medclms_train.csv')
rxclmsdf = pd.read_csv('C:/Users/u1427957/OneDrive - University of Utah/MyWorks_2022/Portfolio-Projects/Humana/data/new_data/rxclms_train.csv')
targetdf_holdout = pd.read_csv("C:/Users/u1427957/OneDrive - University of Utah/MyWorks_2022/Portfolio-Projects/Humana/data/target_holdout.csv")
rxclmsdf_holdout = pd.read_csv("C:/Users/u1427957/OneDrive - University of Utah/MyWorks_2022/Portfolio-Projects/Humana/data/new_data/rxclms_holdout.csv")
medclmsdf_holdout = pd.read_csv("C:/Users/u1427957/OneDrive - University of Utah/MyWorks_2022/Portfolio-Projects/Humana/data/medclms_holdout.csv")

### Original data sets preprocessing

#### Check for percentage Null, blank and missing values in the original datasets

In [3]:
def percentage_null_values(df):
    total_rows = df.shape[0]
    null_counts = df.isnull().sum()
    percentage_null = (null_counts / total_rows) * 100
    return percentage_null

In [4]:
percentage_null_values(rxclmsdf)

therapy_id                 0.000000
document_key               0.000000
ndc_id                     0.000000
service_date               0.000000
process_date               0.000000
pay_day_supply_cnt         0.000000
rx_cost                    0.000000
tot_drug_cost_accum_amt    0.000000
reversal_ind               0.000000
mail_order_ind             0.000000
generic_ind                0.000000
maint_ind                  0.000000
gpi_drug_group_desc        5.909812
gpi_drug_class_desc        5.909812
hum_drug_class_desc        5.909812
strength_meas              6.684717
metric_strength            6.684717
specialty_ind              0.000000
clm_type                   0.000000
dtype: float64

In [5]:
percentage_null_values(medclmsdf)

therapy_id                  0.000000
medclm_key                  0.000000
clm_unique_key              0.000000
primary_diag_cd             0.000000
visit_date                  0.000000
diag_cd2                   24.091694
diag_cd3                   41.292345
diag_cd4                   52.460588
diag_cd5                   61.787757
diag_cd6                   67.264050
diag_cd7                   72.357951
diag_cd8                   75.836420
diag_cd9                   78.718837
process_date                0.000000
reversal_ind               98.713046
pot                         0.000000
util_cat                   43.359059
hedis_pot                   0.000000
clm_type                    0.000000
ade_diagnosis               0.000000
seizure_diagnosis           0.000000
pain_diagnosis              0.000000
fatigue_diagnosis           0.000000
nausea_diagnosis            0.000000
hyperglycemia_diagnosis     0.000000
constipation_diagnosis      0.000000
diarrhea_diagnosis          0.000000
d

In [6]:
medclmsdf = medclmsdf.drop(columns = ['diag_cd4','diag_cd5','diag_cd6','diag_cd7','diag_cd8','diag_cd9', 'reversal_ind'])

In [7]:
medclmsdf.shape

(100159, 20)

In [8]:
percentage_null_values(medclmsdf_holdout)

therapy_id                  0.000000
medclm_key                  0.000000
clm_unique_key              0.000000
primary_diag_cd             0.000000
visit_date                  0.000000
diag_cd2                   23.084539
diag_cd3                   40.517390
diag_cd4                   53.228306
diag_cd5                   64.725379
diag_cd6                   70.170455
diag_cd7                   76.226756
diag_cd8                   78.658747
diag_cd9                   81.516873
process_date                0.000000
reversal_ind               99.104683
pot                         0.000000
util_cat                   43.211949
hedis_pot                   0.000000
clm_type                    0.000000
ade_diagnosis               0.000000
seizure_diagnosis           0.000000
pain_diagnosis              0.000000
fatigue_diagnosis           0.000000
nausea_diagnosis            0.000000
hyperglycemia_diagnosis     0.000000
constipation_diagnosis      0.000000
diarrhea_diagnosis          0.000000
d

In [9]:
medclmsdf_holdout = medclmsdf_holdout.drop(columns = ['diag_cd4','diag_cd5','diag_cd6','diag_cd7','diag_cd8','diag_cd9', 'reversal_ind'])

In [10]:
medclmsdf_holdout.shape

(23232, 20)

#### Drop duplicate therapy_ids from rxclms_train and medclms_train and replace them with central tendency

In [11]:
#Function

def merge_records(df, group_by_column):
    grouped = df.groupby(group_by_column)
    merged_records = []

    for group_name, group_data in grouped:
        numeric_mean = group_data.select_dtypes(include=['int', 'float']).median()
        object_mode = group_data.select_dtypes(include=['object']).mode().iloc[0]

        merged_record = {group_by_column: group_name}
        merged_record.update(numeric_mean)
        merged_record.update(object_mode)

        merged_records.append(merged_record)

    merged_df = pd.DataFrame(merged_records)

    return merged_df

In [12]:
#Apply above function on train set
rxclmsdf = merge_records(rxclmsdf, 'therapy_id')
medclmsdf = merge_records(medclmsdf, 'therapy_id')

In [13]:
print(rxclmsdf.shape)
print(medclmsdf.shape)

(1160, 19)
(536, 20)


In [14]:
#Apply above function on holdout set

rxclmsdf_holdout = merge_records(rxclmsdf_holdout, 'therapy_id')
medclmsdf_holdout = merge_records(medclmsdf_holdout, 'therapy_id')

In [15]:
print(rxclmsdf_holdout.shape)
print(medclmsdf_holdout.shape)

(379, 19)
(185, 20)


#### split the therapy_id column in rxclmsdf and medclmsdf into id,drug_name and therapy_no

In [16]:
#Function

def split_therapy_id_column(df):
    # Create new columns by splitting 'therapy_id' column
    df[['id', 'drug_name', 'therapy_no']] = df['therapy_id'].str.split('-', expand=True)
    
    # Convert 'id' column to integer
    df['id'] = df['id'].astype('int64')
    
    # Drop the original 'therapy_id' column
    df.drop(columns=['therapy_id'], inplace=True)
    
    return df

In [17]:
# Call the function on train data sets

medclmsdf = split_therapy_id_column(medclmsdf)

rxclmsdf = split_therapy_id_column(rxclmsdf)

In [18]:
print(rxclmsdf.shape)
print(medclmsdf.shape)

(1160, 21)
(536, 22)


In [19]:
# Call the function on holdout set data sets 
rxclmsdf_holdout = split_therapy_id_column(rxclmsdf_holdout)

medclmsdf_holdout = split_therapy_id_column(medclmsdf_holdout)

In [20]:
print(rxclmsdf_holdout.shape)
print(medclmsdf_holdout.shape)

(379, 21)
(185, 22)


In [21]:
#Drop the therapy_id column from target_train and target_holdout

In [22]:
targetdf = targetdf.drop(columns = 'therapy_id')

In [23]:
targetdf_holdout = targetdf_holdout.drop(columns = 'therapy_id')

In [24]:
print(targetdf.shape)
print(targetdf_holdout.shape)

(1232, 8)
(420, 7)


#### Merging all 3 datasets

In [25]:
# Merging train data

merged_df1 = pd.merge(targetdf,rxclmsdf, on='id', how='left', suffixes=('_target','_rxclms'))

merged_df2 = pd.merge(merged_df1,medclmsdf, on='id', how='left', suffixes=('_rxclms','_medclms'))

In [26]:
print(merged_df2.shape)
print(merged_df2.isna().sum())

(1232, 49)
id                           0
Total_therapy_days           0
tgt_ade_dc_ind               0
race_cd                     68
est_age                     83
sex_cd                      83
cms_disabled_ind            83
cms_low_income_ind          83
ndc_id                      72
pay_day_supply_cnt          72
rx_cost                     72
tot_drug_cost_accum_amt     72
metric_strength             77
document_key                72
service_date                72
process_date_rxclms         72
reversal_ind                72
mail_order_ind              72
generic_ind                 72
maint_ind                   72
gpi_drug_group_desc         77
gpi_drug_class_desc         77
hum_drug_class_desc         77
strength_meas               77
specialty_ind               72
clm_type_rxclms             72
drug_name_rxclms            72
therapy_no_rxclms           72
medclm_key                 696
clm_unique_key             696
ade_diagnosis              696
seizure_diagnosis          6

In [27]:
# Merging holdout data

merged_df1_holdout = pd.merge(targetdf_holdout,rxclmsdf_holdout, on='id', how='left', suffixes=('_target','_rxclms'))

merged_df2_holdout = pd.merge(merged_df1_holdout,medclmsdf_holdout, on='id', how='left', suffixes=('_rxclms','_medclms'))

In [28]:
print(merged_df2_holdout.shape)
print(merged_df2_holdout.isna().sum())

(420, 48)
id                           0
therapy_start_date           0
race_cd                     18
est_age                     28
sex_cd                      28
cms_disabled_ind            28
cms_low_income_ind          28
ndc_id                      41
pay_day_supply_cnt          41
rx_cost                     41
tot_drug_cost_accum_amt     41
metric_strength             42
document_key                41
service_date                41
process_date_rxclms         41
reversal_ind                41
mail_order_ind              41
generic_ind                 41
maint_ind                   41
gpi_drug_group_desc         42
gpi_drug_class_desc         42
hum_drug_class_desc         42
strength_meas               42
specialty_ind               41
clm_type_rxclms             41
drug_name_rxclms            41
therapy_no_rxclms           41
medclm_key                 235
clm_unique_key             235
ade_diagnosis              235
seizure_diagnosis          235
pain_diagnosis             23

#### Drop the columns which are intuitively not predictors

In [29]:
# Drop columns in merged train set

drop_columns = ['id','Total_therapy_days','metric_strength','document_key','service_date','process_date_rxclms','process_date_medclms','therapy_no_rxclms','medclm_key','clm_unique_key','drug_name_rxclms','visit_date','drug_name_medclms','therapy_no_medclms']
merged_df = merged_df2.drop(columns = drop_columns)

In [30]:
# Drop columns in merged holdout set

drop_columns = ['id','therapy_start_date','metric_strength','document_key','service_date','process_date_rxclms','process_date_medclms','therapy_no_rxclms','medclm_key','clm_unique_key','drug_name_rxclms','visit_date','drug_name_medclms','therapy_no_medclms']

merged_df_holdout = merged_df2_holdout.drop(columns = drop_columns)

In [31]:
print('target : ', merged_df.shape)
print('holdout : ', merged_df_holdout.shape)

target :  (1232, 35)
holdout :  (420, 34)


In [32]:
print(merged_df.info())
print()
print()
print(merged_df_holdout.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1232 entries, 0 to 1231
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tgt_ade_dc_ind           1232 non-null   int64  
 1   race_cd                  1164 non-null   float64
 2   est_age                  1149 non-null   float64
 3   sex_cd                   1149 non-null   object 
 4   cms_disabled_ind         1149 non-null   float64
 5   cms_low_income_ind       1149 non-null   float64
 6   ndc_id                   1160 non-null   float64
 7   pay_day_supply_cnt       1160 non-null   float64
 8   rx_cost                  1160 non-null   float64
 9   tot_drug_cost_accum_amt  1160 non-null   float64
 10  reversal_ind             1160 non-null   object 
 11  mail_order_ind           1160 non-null   object 
 12  generic_ind              1160 non-null   object 
 13  maint_ind                1160 non-null   object 
 14  gpi_drug_group_desc     

#### Replacing missing values (Null, Blank Space, NaN) with new category Unknown and Median

In [33]:
# Replacing missing values - replacing all null, NaN and blank spaces with unknown category for object columns

#Create a function for Unknown category 

def convert_nulls_to_unknown(df):
    # Get a list of object columns
    object_columns = df.select_dtypes(include=['object']).columns
    
    # Replace nulls, blank spaces, and NaNs with 'unknown'
    for col in object_columns:
        df[col].fillna('unknown', inplace=True)
        df[col] = df[col].str.strip().replace('', 'unknown')
    
    return df

In [34]:
# Apply the function on train set
merged_df = convert_nulls_to_unknown(merged_df)

# Apply the function on holdout set
merged_df_holdout = convert_nulls_to_unknown(merged_df_holdout)

In [35]:
# Replacing missing values - replacing all null, NaN and blank spaces with median value for numeric columns

def replace_nulls_with_median(df):
    # Get a list of numeric columns
    numeric_columns = df.select_dtypes(include=['number']).columns
    
    # Replace nulls, blank spaces, and NaNs with median
    for col in numeric_columns:
        median_value = df[col].median()
        df[col].fillna(median_value, inplace=True)
    
    return df

In [36]:
# Apply the function on train set
merged_df = replace_nulls_with_median(merged_df)

# Apply the function on holdout set
merged_df_holdout = replace_nulls_with_median(merged_df_holdout)

In [37]:
print(merged_df.info())
print()
print()
print(merged_df_holdout.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1232 entries, 0 to 1231
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tgt_ade_dc_ind           1232 non-null   int64  
 1   race_cd                  1232 non-null   float64
 2   est_age                  1232 non-null   float64
 3   sex_cd                   1232 non-null   object 
 4   cms_disabled_ind         1232 non-null   float64
 5   cms_low_income_ind       1232 non-null   float64
 6   ndc_id                   1232 non-null   float64
 7   pay_day_supply_cnt       1232 non-null   float64
 8   rx_cost                  1232 non-null   float64
 9   tot_drug_cost_accum_amt  1232 non-null   float64
 10  reversal_ind             1232 non-null   object 
 11  mail_order_ind           1232 non-null   object 
 12  generic_ind              1232 non-null   object 
 13  maint_ind                1232 non-null   object 
 14  gpi_drug_group_desc     

#### Encoding train and holdout datasets

In [ ]:
# Encode train datasets using label encoder

In [38]:
from sklearn.preprocessing import LabelEncoder

In [39]:
# Create a function for label encoding
def convert_object_to_numeric(df):
    # Select columns with object data type
    object_columns = df.select_dtypes(include=['object']).columns

    # Initialize LabelEncoder
    le = LabelEncoder()

    for column in object_columns:
        # Fit and transform the LabelEncoder on each column
        df[column] = le.fit_transform(df[column])
    
    return df

In [40]:
# Call the label encoder function on train and holdout set

merged_df = convert_object_to_numeric(merged_df)

merged_df_holdout = convert_object_to_numeric(merged_df_holdout)

In [42]:
print(merged_df.info())
print()
print()
print(merged_df_holdout.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1232 entries, 0 to 1231
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tgt_ade_dc_ind           1232 non-null   int64  
 1   race_cd                  1232 non-null   float64
 2   est_age                  1232 non-null   float64
 3   sex_cd                   1232 non-null   int32  
 4   cms_disabled_ind         1232 non-null   float64
 5   cms_low_income_ind       1232 non-null   float64
 6   ndc_id                   1232 non-null   float64
 7   pay_day_supply_cnt       1232 non-null   float64
 8   rx_cost                  1232 non-null   float64
 9   tot_drug_cost_accum_amt  1232 non-null   float64
 10  reversal_ind             1232 non-null   int32  
 11  mail_order_ind           1232 non-null   int32  
 12  generic_ind              1232 non-null   int32  
 13  maint_ind                1232 non-null   int32  
 14  gpi_drug_group_desc     

### Modeling

#### Split the train set into train and test

In [43]:
# Assuming 'merged_df_undersampled_encoded' is your DataFrame and 'target' is your target variable
X = merged_df.drop(columns=['tgt_ade_dc_ind'])  # Features
y = merged_df['tgt_ade_dc_ind']  # Target variable

# Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### XGboost with hyperparameter tuned using GridSearch

In [44]:
# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 150],  # Number of boosting rounds
    'max_depth': [3, 4, 5],  # Maximum depth of a tree
    'learning_rate': [0.01, 0.1, 0.2],  # Step size shrinkage to prevent overfitting
    'subsample': [0.8, 1.0],  # Fraction of samples used for fitting the trees
    'colsample_bytree': [0.8, 1.0],  # Fraction of features used for fitting the trees
    'gamma': [0, 1, 5],  # Minimum loss reduction required to make a further partition on a leaf node
    'reg_alpha': [0, 1, 10],  # L1 regularization term on weights
    'reg_lambda': [0, 1, 10],  # L2 regularization term on weights
    'min_child_weight': [1, 3, 5]  # Minimum sum of instance weight needed in a child
    }

# Create an XGBoost classifier
xgb_model = xgb.XGBClassifier(objective='binary:logistic',scale_pos_weight=3)

# Create a GridSearchCV object
gscv = GridSearchCV(xgb_model, param_grid, cv=5, scoring='accuracy')

#Fit the model to the training data
gscv.fit(X_train, y_train)

best_params = gscv.best_params_
print(best_params)

#Get the best estimator from grid search
best_xgb_model = gscv.best_estimator_


{'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.2, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 150, 'reg_alpha': 0, 'reg_lambda': 0, 'subsample': 1.0}


In [45]:
#best_xgb_model = xgb.XGBClassifier(min_samples_leaf = 1, min_samples_split = 2, 
#                                   max_depth = 15, n_estimators = 150, max_features = 'auto')
##fit the model
#best_xgb_model.fit(X_train, y_train)

In [46]:
X_pred = best_xgb_model.predict(X_train)

In [47]:
auc_roc_train = roc_auc_score(y_train, best_xgb_model.predict_proba(X_train)[:, 1])
print(f"AUC-ROC Score of train set: {auc_roc_train}")

AUC-ROC Score of train set: 1.0


In [48]:
# Define the number of folds for cross-validation
num_folds = 10

# Define a custom scoring function using roc_auc_score
def custom_roc_auc(model, X, y):
    y_pred_proba = model.predict_proba(X)[:,1]
    return roc_auc_score(y, y_pred_proba)

# Perform cross-validation
cv_scores = cross_val_score(best_xgb_model, X_train, y_train, cv=num_folds, scoring=custom_roc_auc)


In [49]:
# Display the cross-validation scores
print("Cross-validation scores:", cv_scores)
print()
print(f"Mean AUC: {np.mean(cv_scores)}")

Cross-validation scores: [0.87654321 0.75308642 0.80493827 0.87037037 0.8494382  0.8289638
 0.8289638  0.81772784 0.76654182 0.88639201]

Mean AUC: 0.8282965737272854


In [50]:
y_pred = best_xgb_model.predict(X_test)

In [51]:
auc_roc_test = roc_auc_score(y_test, best_xgb_model.predict_proba(X_test)[:, 1])
print(f"AUC-ROC Score test set : {auc_roc_test}")

AUC-ROC Score test set : 0.8741733379742429


### Predictions on holdout set

In [52]:
holdout_pred = best_xgb_model.predict(merged_df_holdout)

In [53]:
holdout_pred_prob = best_xgb_model.predict_proba(merged_df_holdout)

In [54]:
#holdout_pred_prob

In [55]:
holdout_pred_prob = holdout_pred_prob[:, 0]

In [56]:
#Create a submission DataFrame
# With id column in merged df
#submission_with_Target = pd.DataFrame({
#    "id": merged_df_holdout["id"], 
#    "Target": holdout_pred,
#    "score": holdout_pred_prob
#})

# Without id column in merged df

submission_with_Target = pd.DataFrame({
   "id": targetdf_holdout["id"], 
   "Target": holdout_pred,
   "score": holdout_pred_prob
})

In [57]:
submission_with_Target['Rank'] = submission_with_Target['score'].rank(ascending=False, method='min').astype(int)

submission_with_Target = submission_with_Target.sort_values(by='Rank').reset_index(drop=True)

print('submission shape: ', submission_with_Target.shape)
print()
print('class distribution: ', '\n',submission_with_Target['Target'].value_counts())
print()
print(submission_with_Target)

submission shape:  (420, 4)

class distribution:  
 0    383
1     37
Name: Target, dtype: int64

             id  Target     score  Rank
0    1123008765       0  0.999977     1
1    1108008162       0  0.999937     2
2    1044580345       0  0.999931     3
3    1041799332       0  0.999929     4
4    1117117278       0  0.999901     5
..          ...     ...       ...   ...
415  1007523223       1  0.092871   416
416  1029150179       1  0.087266   417
417  1016317684       1  0.080454   418
418  1078935587       1  0.053380   419
419  1119482706       1  0.049798   420

[420 rows x 4 columns]


In [58]:
#Save the submission_2 DataFrame to a CSV file
submission_with_Target.to_csv("C:/Users/u1427957/OneDrive - University of Utah/MyWorks_2022/Portfolio-Projects/Humana/data/new_data/Humana_10-06-2023_XGBOOST_with_Target_without_id.csv", index=False)

In [59]:
#Create a submission DataFrame
#submission_without_Target = pd.DataFrame({
   # "id": merged_df_holdout["id"], 
   # "score": holdout_pred_prob
#})

# Without id column in merged df

submission_without_Target = pd.DataFrame({
   "id": targetdf_holdout["id"], 
   "score": holdout_pred_prob
})

In [60]:
submission_without_Target['Rank'] = submission_without_Target['score'].rank(ascending=False, method='min').astype(int)

submission_without_Target = submission_without_Target.sort_values(by='Rank').reset_index(drop=True)

print('submission shape: ', submission_without_Target.shape)
print()
#print('class distribution: ', '\n',submission_without_Target['Target'].value_counts())
print()
print(submission_without_Target)

submission shape:  (420, 3)


             id     score  Rank
0    1123008765  0.999977     1
1    1108008162  0.999937     2
2    1044580345  0.999931     3
3    1041799332  0.999929     4
4    1117117278  0.999901     5
..          ...       ...   ...
415  1007523223  0.092871   416
416  1029150179  0.087266   417
417  1016317684  0.080454   418
418  1078935587  0.053380   419
419  1119482706  0.049798   420

[420 rows x 3 columns]


In [61]:
#Save the submission_2 DataFrame to a CSV file
submission_without_Target.to_csv("C:/Users/u1427957/OneDrive - University of Utah/MyWorks_2022/Portfolio-Projects/Humana/data/new_data/Humana_10-06-2023_XGBOOST_without_Target_without_id.csv", index=False)